# Preprocess Payments Dataset

In [6]:
from pathlib import Path

import pandas as pd
import numpy as np

In [7]:
BASE_DIR = Path.cwd().parent
DATA_DIR = (BASE_DIR / "data").resolve()

In [8]:
payments = pd.read_csv(DATA_DIR/"raw"/"olist_order_payments_dataset.csv")
df_payments = pd.DataFrame(payments)

In [9]:
print("\npayments columns:\n", df_payments.columns)


payments columns:
 Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')


In [10]:
print(df_payments['order_id'].value_counts().max())

29


In [11]:
# Con esto conseguimos evitar duplicados al hacer merge de datasets por tener order_id duplicados

payments_total = df_payments.groupby("order_id", as_index=False)["payment_value"].sum()
payments_total.rename(columns={"payment_value": "total_payment"}, inplace=True)

payment_type_main = df_payments.groupby("order_id")["payment_type"].agg(lambda x: x.mode()[0]).reset_index()

max_installments = df_payments.groupby("order_id", as_index=False)["payment_installments"].max()
max_installments.rename(columns={"payment_installments": "max_installments"}, inplace=True)

df_payments = payments_total.merge(payment_type_main, on="order_id", how="left")
df_payments = df_payments.merge(max_installments, on="order_id", how="left")

df_payments.head(3)

,order_id,total_payment,payment_type,max_installments
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,credit_card,2
1,00018f77f2f0320c557190d7a144bdd3,259.83,credit_card,3
2,000229ec398224ef6ca0657da4fc703e,216.87,credit_card,5


Con esto, el dataset *"payments"* tiene order id individuales, de manera que no se harán combinaciones al hacer merge con las demas tablas por producto cartesiano.